In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
%matplotlib inline

In [ ]:
import monox as mx

In [ ]:
train_data_fsr              =  mx.load_pkl_from_targz("./data/train_data_fsr.tar.gz","train_data_fsr.pkl")
test_data_fsr               =  mx.load_pkl_from_targz("./data/test_data_fsr.tar.gz","test_data_fsr.pkl")

# train_data_fsr_log          =  mx.load_pkl_from_targz("./data/train_data_fsr_log.tar.gz","train_data_fsr_log.pkl") # same thing but in log scale
# test_data_fsr_log           =  mx.load_pkl_from_targz("./data/test_data_fsr_log.tar.gz","test_data_fsr_log.pkl")   # same thing but in log scale 

# train_data_fsr_cleaned      =  mx.load_pkl_from_targz("./cleaned_data/train_data_fsr.tar.gz","train_data_fsr.pkl")         # same thing but i keep only data with bout 70% bins filled
# train_data_fsr_log_cleaned  =  mx.load_pkl_from_targz("./cleaned_data/train_data_fsr_log.tar.gz","train_data_fsr_log.pkl") # same thing but i keep only data with bout 70% bins filled, log scale

# Visualize Dataset

A. Data / Input (What you expect to see in experiment) : 
- MET : missing transverse energy
- PT(j[1]) : transverse momentum of leading jet
- ETA(j[1]) : pseudorapidity of leading jet

B. Output (What you are trying to infer) : 
- MASS : Mass of Dark Matter
- COUP : Coupling Between Dark Matter and the Mediator

C. Auxiliary Inputs (Not used in your project, and they should be same/similar for every row in this dataset)<br> 
Cuts: 
- MET_CUT : Minimum Missing Transverse Energy
- PT(j[1])_CUT : Minimum transverse momentum of leading jet
- ETA(j[1])_CUT : Maximum Absolute pseudorapidity of leading jet<br>

Range: 
- PT(j[1])_LOW : lower limit of PT histograms    
- PT(j[1])_UP : upper limit of PT histograms 
- MET_LOW : lower limit of MET histograms    
- MET_UP : upper limit of MET histograms 
- ETA(j[1])_LOW : lower limit of ETA histograms (without the negative)   
- ETA(j[1])_UP : upper limit of ETA histograms <br>

Number of Events: 
- PT(j[1])_NEVENTS : total number of events in the PT histogram
- MET_NEVENTS : total number of events in the MET histogram
- ETA(j[1])_NEVENTS : total number of events in the ETA histogram



In [ ]:
display(train_data_fsr)


## Mass and coupling possibilities in the dataset

In [ ]:
mx.plot_distribution(train_data_fsr)

In [ ]:
mx.plot_distribution(test_data_fsr)

Remark: 
- Its expensive to simulate many events for each mass-coupling parameter space point. 
- So I use data augmentation to get more histogram from each data point. (You can sample from the same amount of events to build different histogram)  
- Datasize means how many histogram you have with that mass-coupling parameter space point

## Number of Signal events 

In [ ]:
xsecpd = pd.read_csv("./xsec.csv")
xsecpd["nevents"] = xsecpd["xsec(pb)"]*139*1000
display(xsecpd)


In [ ]:
plt.scatter(xsecpd["mass"], xsecpd["coup"],c=np.log10(xsecpd["nevents"]),)
plt.xlabel("mass")
plt.ylabel("coupling")
plt.colorbar(label="log10(number of SIGNAL events)")
plt.show()

Remarks : The number events gets lower with lower couplings and higher mass

# Example histograms

In [ ]:
PT_LOW   = (train_data_fsr["PT(j[1])_LOW"].unique())[0]
PT_UP    = (train_data_fsr["PT(j[1])_UP"].unique())[0]
MET_LOW  = (train_data_fsr["MET_LOW"].unique())[0]
MET_UP   = (train_data_fsr["MET_UP"].unique())[0]
ETA_LOW  = (train_data_fsr["ETA(j[1])_LOW"].unique())[0]
ETA_UP   = (train_data_fsr["ETA(j[1])_UP"].unique())[0]


In [ ]:
# Randomly sample a data point
dat = train_data_fsr.sample(1)

# Create figure and subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# PT plot
bin_edges = np.linspace(PT_LOW, PT_UP, 31)
bin_centers = np.diff(bin_edges) / 2 + bin_edges[:-1]
axes[0].bar(bin_centers, dat["PT(j[1])"].values[0], width=np.diff(bin_edges), 
            label=f"Mass={dat['MASS'].values[0]} GeV, Coup= {dat['COUP'].values[0]}")
axes[0].hlines(1, PT_LOW, PT_UP, color="r")
axes[0].set_yscale("log")
axes[0].set_ylim(0.9, 10 * max(dat["PT(j[1])"].values[0]))
axes[0].set_xlabel("PT")
axes[0].set_ylabel("Number of Events")
#axes[0].legend()

# MET plot
bin_edges = np.linspace(MET_LOW, MET_UP, 31)
bin_centers = np.diff(bin_edges) / 2 + bin_edges[:-1]
axes[1].bar(bin_centers, dat["MET"].values[0], width=np.diff(bin_edges), 
            label=f"Mass={dat['MASS'].values[0]} GeV, Coup= {dat['COUP'].values[0]}")
axes[1].hlines(1, MET_LOW, MET_UP, color="r")
axes[1].set_yscale("log")
axes[1].set_ylim(0.9, 10 * max(dat["MET"].values[0]))
axes[1].set_xlabel("MET")
axes[1].set_ylabel("Number of Events")
#axes[1].legend()

# ETA plot
bin_edges = np.linspace(-ETA_LOW, ETA_UP, 31)
bin_centers = np.diff(bin_edges) / 2 + bin_edges[:-1]
axes[2].bar(bin_centers, dat["ETA(j[1])"].values[0], width=np.diff(bin_edges), 
            label=f"Mass={dat['MASS'].values[0]} GeV, Coup= {dat['COUP'].values[0]}")
axes[2].hlines(1, -ETA_LOW, ETA_UP, color="r")
axes[2].set_yscale("log")
axes[2].set_ylim(0.9, 10 * max(dat["ETA(j[1])"].values[0]))
axes[2].set_xlabel("ETA")
axes[2].set_ylabel("Number of Events")
#axes[2].legend()
plt.suptitle(f"Mass={dat['MASS'].values[0]} GeV, Coup= {dat['COUP'].values[0]}")
# Adjust layout
plt.tight_layout()
plt.show()

Remarks : 
- Note that PT and MET looks very similar (Because of conservation of momentum)
- Simulated data gets more noisy as it goes to higher value of MET or PT. 
- Probablility of generating such events gets lower 
- Similar to the probability of finding them in experiments
- Also, It becomes more computationally difficult to simulate


### Compare different data points

In [ ]:
xdat = train_data_fsr.sample(50)
bin_edges = np.linspace(PT_LOW, PT_UP,31)
bin_centers = np.diff(bin_edges) / 2 + bin_edges[:-1]
for _, dat in xdat.iterrows():
    plt.bar(bin_centers, dat["PT(j[1])"],width=np.diff(bin_edges),label=f"Mass={dat['MASS']} GeV, Coup= { dat['COUP']}",alpha=0.25)
plt.hlines(1, PT_LOW, PT_UP,color="r")
plt.yscale("log")
plt.ylim(0.9,10*max(dat["PT(j[1])"]))
plt.xlabel("PT")
plt.ylabel("Number of Events")
plt.show()

bin_edges = np.linspace(MET_LOW, MET_UP,31)
bin_centers = np.diff(bin_edges) / 2 + bin_edges[:-1]
for _, dat in xdat.iterrows():
     plt.bar(bin_centers, dat["MET"],width=np.diff(bin_edges),label=f"Mass={dat['MASS']} GeV, Coup= { dat['COUP']}",alpha=0.25)
plt.hlines(1, PT_LOW, PT_UP,color="r")
plt.yscale("log")
plt.ylim(0.9,10*max(dat["MET"]))
plt.xlabel("MET")
plt.ylabel("Number of Events")
plt.show()

bin_edges = np.linspace(-ETA_LOW, ETA_UP,31)
bin_centers = np.diff(bin_edges) / 2 + bin_edges[:-1]
for _, dat in xdat.iterrows():
    plt.bar(bin_centers, dat["ETA(j[1])"],width=np.diff(bin_edges),label=f"Mass={dat['MASS']} GeV, Coup= { dat['COUP']}",alpha=0.25)
plt.hlines(1, -ETA_LOW, ETA_UP,color="r")
plt.yscale("log")
plt.ylim(0.9,10*max(dat["ETA(j[1])"]))
plt.xlabel("ETA")
plt.ylabel("Number of Events")
plt.show()


Remark: you can see all the slight difference between data points